![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true)
***
# *Practicum AI:* CNN - Cat / Dog

This exercise adapted from Baig et al. (2020) <i>The Deep Learning Workshop</i> from <a href="https://www.packtpub.com/product/the-deep-learning-workshop/9781839219856">Packt Publishers</a> (Exercise 3.03, page 130).

(20 Minutes)

#### Introduction
In this exercise, we will create a deep learning convolutional neural network model to classify cats and dogs.  Actually, this is the first exercise in a two-part sequence.  In next week's workshop, you will increase the size of the training dataset, in a process called **data augmentation**.

<p float="left">
  <img src="images/02.2_cat_image.jpg" width="250" height="250" />
  <img src="images/02.2_dog_image.jpg" width="225" height="225"/> 
</p>

#### 1. Import packages for the notebook

In [1]:
import pathlib
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

#### 2. Create a link to the dataset

In [2]:
file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Exercise3.03/cats_and_dogs_filtered.zip'

#### 3. Download the dataset

Download the dataset to the data folder. After running this cell, explore the `data/datasets/cats_and_dogs_filtered folder`.

In [3]:
zip_dir = tf.keras.utils.get_file('cats_and_dogs.zip', origin = file_url, extract = True, cache_dir='data')

#### 4. Create a path variable to cats_and_dogs_filtered directory

In [4]:
path = os.path.join(pathlib.Path(zip_dir).parent, 'cats_and_dogs_filtered')

#### 5. Create path variables to the train and validation directories

In [5]:
train_dir = os.path.join(path, 'train')
validation_dir = os.path.join(path, 'validation')

#### 6. Create path variables to the other directories

In [6]:
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

#### 7. Create variables to hold train & validation image counts

In [7]:
total_train = len(os.listdir(train_cats_dir)) + len(os.listdir(train_dogs_dir))
total_val = len(os.listdir(validation_cats_dir)) + len(os.listdir(validation_dogs_dir))

#### 8. Instantiate two ImageDataGenerator classes

Note: The data in these two objects will be rescaled (standardized).

In [8]:
train_image_generator = ImageDataGenerator(rescale = 1./255)
validation_image_generator = ImageDataGenerator(rescale = 1./255)

#### 9. Assign values to three parameter variables

In [9]:
batch_size = 16
img_height = 100
img_width  = 100

#### 10. Create the training data generator

Note that we are using the `image_generator` here to get the images from the folders. No data augmentation is being done at this stage. We will do that in our next session.

In [10]:
train_data_gen = train_image_generator.flow_from_directory(batch_size  = batch_size,
                                                           directory   = train_dir,
                                                           shuffle     = True,
                                                           target_size = (img_height, img_width),
                                                           class_mode  = 'binary')

Found 2000 images belonging to 2 classes.


#### 11. Create the validation data generator

In [11]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size  = batch_size,
                                                              directory   = validation_dir,
                                                              target_size = (img_height, img_width),
                                                              class_mode  = 'binary')

Found 1000 images belonging to 2 classes.


#### 12. Set random seeds for repeatability

In [12]:
np.random.seed(8)
tf.random.set_seed(8)

#### 13. Create our model

* The model starts with a convolutional layer. In this layer, we feature 64 3x3 kernels. We define the imput shape with the image dimensions set above and three chanels for color images.
* Next, there is a MaxPooling layer
* Followed by another convolutional layer, this time with 126 kernels to learn.
* Another MaxPooling layer
* Then we flatten the images into a long vector for a Dense layer with 128 neurons
* That feeds into the final layer with a single neuron and a sigmoid activation--this is a binary classification (cat or dog)

In [14]:
model = tf.keras.Sequential([
    layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width ,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

#### 14. Set the omptimizer and learning rate

In [15]:
optimizer = tf.keras.optimizers.Adam(0.001)

#### 15. Compile the model

In [16]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#### 16. Display the model summary

Notice the numbers of parameters. A single dense layer for a 100X100X3 image would have 3001 parameters, yet the CNN convolutional layer, even with 64 kernels has only 1,792! The vast majority of our parameters still come from the dense layer. 

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 67712)             0         
                                                                 
 dense_2 (Dense)             (None, 128)              

#### 17. Train the model

In [18]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5


2022-09-14 14:58:42.704571: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-09-14 14:58:43.458746: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2022-09-14 14:58:43.495018: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-09-14 14:58:44.390833: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


125/125 [==============================] - 10s 58ms/step - loss: 0.7204 - accuracy: 0.5275 - val_loss: 0.6778 - val_accuracy: 0.5817
Epoch 2/5
125/125 [==============================] - 7s 56ms/step - loss: 0.6727 - accuracy: 0.6025 - val_loss: 0.6529 - val_accuracy: 0.6431
Epoch 3/5
125/125 [==============================] - 7s 56ms/step - loss: 0.6307 - accuracy: 0.6470 - val_loss: 0.6741 - val_accuracy: 0.5887
Epoch 4/5
125/125 [==============================] - 7s 57ms/step - loss: 0.5593 - accuracy: 0.7130 - val_loss: 0.7072 - val_accuracy: 0.6542
Epoch 5/5
125/125 [==============================] - 7s 56ms/step - loss: 0.4399 - accuracy: 0.7815 - val_loss: 0.7167 - val_accuracy: 0.6522


Not too bad! Our fairly simple model can classify cats and dogs with an accuracy of about 80% on the traning data and almost 70% on the validation data after only 5 epochs. The much lower validation accuracy is an indication of overfitting. Remember there are only 2000 images total in the training dataset.

***
## *Practicum AI:* CNN - Cat/Dog Data Augmentation

As is often the case in reality, we are limited in the amount of data for model training. To help get around this limitation, we rely on **data augmentation** to generate transformations of our training data, to create "new" imges to train on.  By rotating, flipping, color shifting, etc. our original images, we can increase the diversity of images of cats and dogs that our model sees and improve its performance without needing to get more images to train on.

When applying data augmentation, it is important to consider what augmentations are or are not appropriate for your use case. For example, a classifier for red roses and blue violets might not do as well if we employ color shifting in the augmentation!

#### 18. Define new data generators

We only do data augmenation on training data. Here, we redefine the ImageDataGenerator in step 8 to include data augmentation on the training data, but keep the same ImageDataGenerator used above for the validation_image_generator.

In [19]:
train_image_generator  = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range = 20,      # Randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1,         # Randomly zoom image
    width_shift_range = 0.1,  # Randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.1, # Randomly shift images vertically (fraction of total height)
    horizontal_flip =True,    # Randomly flip images horizontally
    vertical_flip =  False,   # Don't randomly flip images vertically
)  

validation_image_generator  = ImageDataGenerator(rescale = 1./255)

#### 19 Setup the new data generators

In [20]:
train_data_gen = train_image_generator.flow_from_directory(batch_size  = batch_size,
                                                           directory   = train_dir,
                                                           shuffle     = True,
                                                           target_size = (img_height, img_width),
                                                           class_mode  = 'binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size  = batch_size,
                                                              directory   = validation_dir,
                                                              target_size = (img_height, img_width),
                                                              class_mode  = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


#### 20. Reset the model

In [21]:
np.random.seed(8)
tf.random.set_seed(8)

model = tf.keras.Sequential([
    layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width ,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(0.001)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#### 21. Display the model summary

```python
model.summary()
```


In [ ]:
# Code it!

#### 22. Train the model

```python
model.fit(
    train_data_gen,
    steps_per_epoch = total_train // batch_size,
    epochs = 5,
    validation_data = val_data_gen,
    validation_steps = total_val // batch_size
)
```

In [ ]:
# Code it!